### Install Dependencies

In [1]:
!pip install transformers
!pip install evaluate
!pip install rouge
!pip install langchain
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install -U nlp

!pip install evaluate
!pip install dill==0.3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 946.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: dill
   

In [2]:
import torch
import json
import torch.nn as nn
import nlp
import pandas as pd
import numpy as np
import transformers
import evaluate
import dataclasses

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import warnings
warnings.filterwarnings("ignore")

### Downloading the Model

In [3]:
# text2text model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('declare-lab/flan-alpaca-base')  # 248 M
model = AutoModelForSeq2SeqLM.from_pretrained("declare-lab/flan-alpaca-base")

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [4]:
# Save the model's parameters to a file
model_id = "model_large"                             # give your model name to avoid confusion

model_path = model_id
torch.save(model.state_dict(), model_path)

# Check the size of the saved model file
import os
model_file_size_bytes = os.path.getsize(model_path)
model_file_size_mb = model_file_size_bytes / (1024 * 1024)  # Convert to MB
model_file_size_gb = model_file_size_mb / 1024  # Convert to GB

print(f"Model Size: {model_file_size_gb:.4f} GB")

Model Size: 0.9224 GB


### Preprocessing the dataset (SQUAD)

In [5]:
# Function to add end-of-sequence (EOS) tokens to the input and target texts in the example
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question'], example['context'])
    example['target_text'] = '%s </s>' % example['answers']['text'][0]
    return example

# Function to tokenize a batch of examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=16)

    # Create a dictionary containing input and target encodings
    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [7]:
# load train and validation split of squad
train_dataset  = nlp.load_dataset('squad', split=nlp.Split.TRAIN)
valid_dataset = nlp.load_dataset('squad', split=nlp.Split.VALIDATION)

train_dataset = train_dataset.select(range(30000))
valid_dataset = valid_dataset.select(range(30000))

# map add_eos_to_examples function to the dataset example wise
train_dataset = train_dataset.map(add_eos_to_examples)
# map convert_to_features batch wise
train_dataset = train_dataset.map(convert_to_features, batched=True)

valid_dataset = valid_dataset.map(add_eos_to_examples, load_from_cache_file=False)
valid_dataset = valid_dataset.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/408a8fa46a1e2805445b793f1022e743428ca739a34809fce872f0c7f17b44ab. Subsequent calls will reuse this data.


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30000 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
len(train_dataset), len(valid_dataset)

(30000, 10570)

In [9]:
@dataclass  #dataclass decorator for creating data classes
class T2TDataCollator():# Callable class that takes a batch of samples from a Dataset, combines them into a batch, and returns a dictionary of tensors
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:   # Callable class that takes a batch of samples from a Dataset, combines them into a batch, and returns a dictionary of tensors

    # Stack the tensors from the batch for language model labels
    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100       # Replace 0s in lm_labels with -100 to ignore loss computation for padding tokens
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': lm_labels,
        'decoder_attention_mask': decoder_attention_mask
    }

### Training the model

In [10]:
!pip install rouge_score
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute evaluation metrics using the specified metric and post-processed texts
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}

    # Calculate the average length of generated sequences
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=802afb465aa2f9f7ac51e4a4b2895b736944814b5936831f6a14d21e6bbf8b14
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


# Define training args
training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        num_train_epochs=1,
        learning_rate=3e-4,
        fp16=False,
        logging_steps=100,
        output_dir="outputs",

)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),

)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!



In [14]:
trainer.train().metrics

Step,Training Loss
100,0.006300
200,0.010600
300,0.009500
400,0.007400
500,0.008000
600,0.006000
700,0.006100
800,0.005800
900,0.004500
1000,0.005600


Step,Training Loss
100,0.006300
200,0.010600
300,0.009500
400,0.007400
500,0.008000
600,0.006000
700,0.006100
800,0.005800
900,0.004500
1000,0.005600


{'train_runtime': 9156.773,
 'train_samples_per_second': 3.276,
 'train_steps_per_second': 0.41,
 'total_flos': 2.054272057344e+16,
 'train_loss': 0.0034704766472180683,
 'epoch': 1.0}

### Evalution

In [16]:
DEVICE = "cuda:0"
Q_LEN = 256

def get_answer(question, context):   # Function to obtain an answer to a given question and context using the pre-trained Seq2Seq model
  inputs = tokenizer(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)  # Tokenize the input question and context, ensuring max length, padding, and truncation

  # Convert tokenized inputs to PyTorch tensors and move to the specified device
  input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
  attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

  outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)


  return tokenizer.decode(outputs[0], skip_special_tokens=True) # Decode the generated output, skipping special tokens

context = "Manuel have created RuPERTa-base with the support of HF-Transformers and Google"
question = "Who has supported Manuel?"

get_answer(question, context)

'question has supported Manuel? Manuel have created RuPERTa-base with the support of '

In [ ]:
questions = ["What is the Invoice number?",
             "What is the Invoice date?",
             "What is the total amount?",
             "State the From address of Invoice?",
             "State the To address of Invoice?",
             "What is the order number?"
             ]

context = "[Document(page_content='Invoice\n\nInvoice Number\n\nINV-3337\n\nFrom: DEMO - Sliced Invoices Suite 5A-1204 123 Somewhere Street Your City AZ 12345 admin@slicedinvoices.com\n\nOrder Number\n\n12345\n\nInvoice Date\n\nJanuary 25, 2016\n\nDue Date\n\nJanuary 31, 2016\n\nTotal Due\n\n$93.50\n\nTo: Test Business 123 Somewhere St Melbourne, VIC 3000 test@test.com\n\nP aid\n\nHrs/Qty\n\nService\n\nRate/Price\n\nAdjust\n\nSub Total\n\nWeb Design This is a sample description...\n\n1.00\n\n$85.00\n\n0.00%\n\n$85.00\n\nSub Total\n\n$85.00\n\nTax\n\n$8.50\n\nTotal\n\n$93.50\n\nANZ Bank ACC # 1234 1234 BSB # 4321 432\n\nPayment is due within 30 days from date of invoice. Late payment is subject to fees of 5% per month. Thanks for choosing DEMO - Sliced Invoices | admin@slicedinvoices.com Page 1/1', metadata={'source': 'invoice.pdf'})]"
for qn in questions:
  print(get_answer(qn,context))


12345 admin@slicedinvoices.com Order Number 12345 Invoice Date
12345 Invoice Date January 25, 2016 Due Date January 31, 2016 Total Due $93.50
12345 Invoice Date January 25, 2016 Due Date January 31, 2016 Total Due $93.50
12345 admin@slicedinvoices.com Order Number 12345 Invoice Date
[Document(page_content='Invoice Invoice Number INV-33
[Document(page_content='Invoice Invoice Number INV-33


In [ ]:
questions = ["What is the capital of France?",
             "How many days are there I a week?",
             "What is the largest planet in our solar system?",
             "What is the tallest mountain in the world?",
             "What is the main language in India?",
             "Who is the author of Harry Potter book series?"
             ]

context = " "
for qn in questions:
  print(get_answer(qn,context))

The capital of France is Paris.
There are a few days in a week.
The largest planet in our solar system is Mercury. Mercury is the second largest planet in the Solar
The tallest mountain in the world is Mount Everest, located in the Tibetan Plateau.
The main language in India is Hindi.
The author of Harry Potter book series is Harry Potter and the Philosopher's Stone.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [15]:
!pip install pynvml
import psutil
import pynvml


# RAM Usage
ram = psutil.virtual_memory()
ram_usage = ram.used / (1024 * 1024 * 1024)
print("RAM Usage:", ram_usage, "GB")
print()

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:',(round(torch.cuda.memory_allocated(0)/1024**3,1) + (round(torch.cuda.memory_cached(0)/1024**3,1))), 'GB')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
print()

# Disk Usage
disk_usage = psutil.disk_usage('/')
print("Disk Usage:", (disk_usage.used / (1024 * 1024 * 1024)-24.2), "GB")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.4 MB/s eta 0:00:00
RAM Usage: 3.6512527465820312 GB

Using device: cuda
Tesla T4
Memory Usage: 15.899999999999999 GB
Allocated: 4.7 GB
Cached:    11.2 GB

Disk Usage: 24.972714233398438 GB
